## Pipeline 2: Search

### Load Packages

In [1]:
!pwd

/Users/alexcheng/Dropbox/GitHub/ga_repo/GA_Portfolio/project-06_Pseudo_Wikipedia_API/ipynb


In [2]:
from os import chdir
chdir('../') # make sure you only run this once

In [4]:
import pandas as pd
import psycopg2
import lib.database_module as db
import lib.encoding_module as enc
import lib.wiki_module as wiki
from lib.encoding_module import get_searchterm_vector, get_page_vector

In [4]:
from sklearn.externals import joblib
transformer = joblib.load('/Users/alexcheng/Desktop/vectorizer.pkl')

In [5]:
db.connect_to_postgres()

Connected to server joshuacook.me.


(<connection object at 0x1166fc050; dsn: 'user=alex password=xxxxx people captain cell' host=joshuacook.me port=5432 dbname=project_6', closed: 0>,
 <cursor object at 0x115f9b240; closed: 0>)

### Parse: Encode search as document vector

In [6]:
search_doc = 'dill pickles'

In [7]:
search_vector_doc = get_searchterm_vector(transformer, search_doc)
search_vector_doc

{'dill pickles': array([[  8.27642998e-04,   2.33422514e-03,   8.63102698e-04,
          -1.62603238e-03,   8.96590508e-03,   1.08725309e-02,
           7.93898716e-04,  -2.60154144e-03,  -8.24504906e-04,
           7.84486216e-04,  -6.13156870e-03,  -1.10114397e-03,
           2.04490520e-03,  -6.46886856e-04,  -3.06195163e-04,
           1.28792896e-04,  -6.27257687e-05,  -3.70461628e-04,
          -1.44502106e-05,  -1.31442484e-04,   3.13528068e-04,
           1.89499743e-04,  -2.73156072e-04,  -1.45686685e-04,
          -1.67707379e-04,  -3.25918484e-04,  -2.44556555e-04,
          -3.94564646e-05,   9.38299306e-05,   2.45228315e-04,
          -3.81661023e-04,   4.95837685e-04,   3.43037966e-04,
           2.20319937e-05,   4.64981697e-04,   7.51851944e-04,
           2.22531176e-05,  -1.27575043e-03,  -2.94506836e-04,
          -2.08674525e-03,   5.83078490e-04,  -1.91211012e-03,
           2.58380789e-05,   6.12018465e-04,  -5.23072800e-03,
           6.45764333e-03,  -2.44615289

### Mine: Select document vectors for all pages from database

In [8]:
page_vectors = db.select_all_page_vectors()

Connected to server joshuacook.me.


In [9]:
indices = [tup[0] for tup in page_vectors]
vectors = [tup[1] for tup in page_vectors]

In [10]:
page_vectors_df = pd.DataFrame(vectors, index=indices)

In [11]:
page_vectors_df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
4341789,0.092992,-0.040985,0.006952,0.000243,0.001681,0.004999,0.003430,0.003523,-0.005129,-0.001911,...,-0.021003,-0.039528,0.027716,0.021033,-0.034768,-0.022294,0.031537,-0.009710,-0.010331,0.025944
48201744,0.021301,0.027559,-0.003917,-0.010877,0.013055,0.000934,0.000128,0.010317,0.001444,-0.009792,...,-0.055040,0.000110,0.057817,0.005156,0.058477,0.008807,0.043527,-0.011693,0.064738,0.051296
2514975,0.124983,0.153025,0.026264,-0.257191,-0.210183,0.074847,0.003862,-0.009666,-0.065705,-0.280972,...,-0.022703,0.015894,-0.002476,0.004992,-0.006653,-0.017394,-0.008525,0.017188,-0.001596,0.000120
35135520,0.191357,-0.039003,-0.006018,0.006389,-0.001106,-0.004748,0.015829,0.005612,0.011846,0.028248,...,0.000578,-0.004204,0.030916,0.010484,-0.012454,0.026050,0.038744,0.015332,-0.008053,0.009506
27303975,0.034292,0.039499,-0.006095,-0.015156,0.026112,-0.039749,-0.081440,-0.036546,0.005076,0.018881,...,0.030056,-0.026965,-0.002426,0.042282,0.024082,0.004498,-0.009517,-0.005315,-0.003784,-0.010202


In [12]:
lookup_df = pd.DataFrame(page_vectors_df.index)
lookup_df.head()

,0
0,4341789
1,48201744
2,2514975
3,35135520
4,27303975


### Model: Find five most similar documents based on document vectors

In [13]:
from sklearn.neighbors import NearestNeighbors

In [14]:
this_NN = NearestNeighbors()

In [15]:
this_NN.fit(page_vectors_df)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [16]:
search_vector_doc['dill pickles']

array([[  8.27642998e-04,   2.33422514e-03,   8.63102698e-04,
         -1.62603238e-03,   8.96590508e-03,   1.08725309e-02,
          7.93898716e-04,  -2.60154144e-03,  -8.24504906e-04,
          7.84486216e-04,  -6.13156870e-03,  -1.10114397e-03,
          2.04490520e-03,  -6.46886856e-04,  -3.06195163e-04,
          1.28792896e-04,  -6.27257687e-05,  -3.70461628e-04,
         -1.44502106e-05,  -1.31442484e-04,   3.13528068e-04,
          1.89499743e-04,  -2.73156072e-04,  -1.45686685e-04,
         -1.67707379e-04,  -3.25918484e-04,  -2.44556555e-04,
         -3.94564646e-05,   9.38299306e-05,   2.45228315e-04,
         -3.81661023e-04,   4.95837685e-04,   3.43037966e-04,
          2.20319937e-05,   4.64981697e-04,   7.51851944e-04,
          2.22531176e-05,  -1.27575043e-03,  -2.94506836e-04,
         -2.08674525e-03,   5.83078490e-04,  -1.91211012e-03,
          2.58380789e-05,   6.12018465e-04,  -5.23072800e-03,
          6.45764333e-03,  -2.44615289e-03,  -8.99000265e-03,
        

In [17]:
NN_results = this_NN.kneighbors(search_vector_doc['dill pickles'])
NN_results

(array([[ 0.26870117,  0.36595974,  0.38815268,  0.39770463,  0.40062235]]),
 array([[1172,  247,  795,  566,  483]]))

In [18]:
NN_results_list = NN_results[1].tolist()
NN_results_list

[[1172, 247, 795, 566, 483]]